In [1]:
from classifier import keyboard, classifier
from scipy.optimize import curve_fit
import numpy as np

red, green, blue = '#ff0040', '#00aa00', '#187bff'
bistroke_times = {}
bistroke_freq = {}
bigrams = set()
kb = keyboard()
key_classifier = classifier(kb)

with open(f"nstrokes/1-skip.txt") as file:
    for l in file:
        layout, bigram, freq, *times = l.split("\t")

        if bigram not in bigrams:
            bigrams.add(bigram)
            
        bistroke_freq[(layout, bigram)] = int(freq)
        bistroke_times[(layout, bigram)] = [
            list(map(int, t.strip()[1:-1].split(", "))) for t in times
        ]

In [2]:
layout = "qwerty"

freqs = []
times = []
is_sfb = []
home_row = []
not_homerow = []
delta_x = []
is_bottom = []
top1 = []
home1 = []
bottom1 = []
top2 = []
home2 = []
bottom2 = []
finger1 = []
finger2 = []
same_hand = []
is_pinky1 = []
is_pinky2 = []
is_ring1 = []
is_ring2 = []
is_middle1 = []
is_middle2 = []
is_index1 = []
is_index2 = []
is_lateral = []
dists = []
actual_times = []
y_dist = []
c = []
labels = []

def get_iqr_avg(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    new_data = [x for x in data if x >= lower_bound and x <= upper_bound]
    print(len(data), len(new_data))
    print(sum(data)/len(data)-sum(new_data)/len(new_data))
    
    return sum(new_data)/len(new_data)


for bg in bigrams:
    if not any([c in '!@#$%^&*()QWERTYUIOP{}|ASDFGHJKL:"ZXCVBNM<>?' for c in bg]):
        if (layout, bg) in bistroke_freq:
            time_data = [t[1] for t in bistroke_times[(layout, bg)] if t[0] > 0]
            actual_times.append(time_data)
            x_val = bistroke_freq[(layout, bg)]
            y_val = get_iqr_avg(time_data)
            
            if x_val != 0:
                freqs.append(x_val)
                times.append(y_val)
                is_sfb.append(int(key_classifier.same_finger(bg)))
                top1.append(key_classifier.kb.get_row(bg[0]) == 3)
                home1.append(key_classifier.kb.get_row(bg[0]) == 2)
                bottom1.append(key_classifier.kb.get_row(bg[0]) == 1)
                top2.append(key_classifier.kb.get_row(bg[1]) == 3)
                home2.append(key_classifier.kb.get_row(bg[1]) == 2)
                bottom2.append(key_classifier.kb.get_row(bg[1]) == 1)
                y_dist.append(key_classifier.get_dy(bg))
                delta_x.append(key_classifier.get_dx(bg))
                same_hand.append(key_classifier.same_hand(bg))
                labels.append(bg)

                # Finger stuff
                finger1.append(abs(key_classifier.kb.get_col(bg[0])))
                finger2.append(abs(key_classifier.kb.get_col(bg[1])))
                
                is_pinky1.append(abs(key_classifier.kb.get_col(bg[0])) == 5)
                is_pinky2.append(abs(key_classifier.kb.get_col(bg[1])) == 5)
                is_ring1.append(abs(key_classifier.kb.get_col(bg[0])) == 4)
                is_ring2.append(abs(key_classifier.kb.get_col(bg[1])) == 4)
                is_middle1.append(abs(key_classifier.kb.get_col(bg[0])) == 3)
                is_middle2.append(abs(key_classifier.kb.get_col(bg[1])) == 3)
                is_index1.append(abs(key_classifier.kb.get_col(bg[0])) in (1,2))
                is_index2.append(abs(key_classifier.kb.get_col(bg[1])) in (1,2))
                is_lateral.append(abs(key_classifier.kb.get_col(bg[1])) == 1)

                c.append(red if is_sfb[-1] else blue if same_hand[-1] else green)

3904 3575
62.33028638369825
3833 3490
75.31689467951742
91667 84373
59.26330130110472
7341 6829
58.87647643389795
44433 41281
61.34194756016899
43288 39472
61.60030417122351
610 583
42.506191828585884
34155 30852
55.35575899184289
77769 71134
69.52392133106889
64966 59684
59.478185396056176
3159 2932
72.63394059805307
6751 6287
74.40975385722447
3651 3463
73.12578272971064
5564 5182
70.1963434645337
34412 31755
60.352894797659076
22272 20581
86.40105412357929
31470 28782
70.70514889913386
13291 12209
70.27508803716609
2269 2140
61.729189852666764
6352 5927
68.87564937852181
47676 44074
66.46951647417927
9469 8831
54.280492510356794
14980 13840
48.5719092701637
57405 53229
66.19236095532187
4496 4186
60.97927095146849
654 620
64.00762059780999
2970 2748
70.23644229779603
67943 61859
66.47117901641076
552 521
64.1039702077943
42003 38878
70.48106664086765
37242 34695
45.59403651554169
12262 11291
75.03299625868476
172274 159476
58.555709727123144
16996 15817
58.95761545252668
65375 60508

In [3]:
import numpy as np

freqs, times, is_lateral, is_sfb, delta_x, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2, c, labels = zip(*sorted(zip(freqs, times, is_lateral, is_sfb, delta_x, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2, c, labels), key = lambda x: x[0]))

freqs = np.array(freqs, dtype=np.float64)
times = np.array(times, dtype=np.float64)
is_sfb = np.array(is_sfb, dtype=np.float64)
delta_x = np.array(delta_x, dtype=np.float64)
top1 = np.array(top1, dtype=np.float64)
home1 = np.array(home1, dtype=np.float64)
bottom1 = np.array(bottom1, dtype=np.float64)
top2 = np.array(top2, dtype=np.float64)
home2 = np.array(home2, dtype=np.float64)
bottom2 = np.array(bottom2, dtype=np.float64)
finger1 = np.array(finger1, dtype=np.float64)
finger2 = np.array(finger2, dtype=np.float64)
is_pinky1 = np.array(is_pinky1, dtype=np.float64)
is_pinky2 = np.array(is_pinky2, dtype=np.float64)
is_ring1 = np.array(is_ring1, dtype=np.float64)
is_ring2 = np.array(is_ring2, dtype=np.float64)
is_middle1 = np.array(is_middle1, dtype=np.float64)
is_middle2 = np.array(is_middle2, dtype=np.float64)
is_index1 = np.array(is_index1, dtype=np.float64)
is_index2 = np.array(is_index2, dtype=np.float64)
dists = np.array(dists, dtype=np.float64)
y_dist= np.array(y_dist, dtype=np.float64)
is_lateral = np.array(is_lateral, dtype=np.float64)
same_hand= np.array(same_hand, dtype=np.float64)

input_data = [freqs, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, is_sfb, is_lateral, delta_x, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2]
input_data = np.array(input_data)

In [ ]:
# def model_function(x, a, b, c, d, e, f, g, h, i, j, k, l):
#     freq_pen = (a * np.log(x[0] + b) + c)
#     finger_pen = (d*x[1]+e*x[2]+f*x[3]+g*x[4] + h)
#     
#     return freq_pen*finger_pen
#     return (a * np.log(x[0] + b) + c) * (d * x[1] + e * x[2] + g * x[3] + f)

# x v. y size and color coding
# bar graph of parameters

# Unit scaling

# freqs, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, is_sfb, delta_x, y_dist, same_hand, bottom2, home2, top2
def model_function(features, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, p24, p25, p26, p27, p28, p29, p30,
                   p31, p32, p33, p34, p35, p36, p37, p38, p39, p40, p41, p42, p43, p44, p45, p46, p47, p48, p49, p50, p51, p52):
    freq, is_sfb, is_lateral, dx, dy, same_hand = features[0], *features[9:14]
    
    #input_data = [freqs, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, is_sfb, is_lateral, delta_x, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2]

    bottom1, home1, top1, bottom2, home2, top2 = features[14:20]
    is_pinky1, is_ring1, is_middle1, is_index1 = features[1:5]
    is_pinky2, is_ring2, is_middle2, is_index2 = features[5:9]

    freq_pen = (p1 * np.log(freq + p2) + p3)
    
    # SHB weighting
    shb_finger1_row_pen = p4*bottom1 + p5*home1 + p6*top1
    shb_finger2_row_pen = p7*bottom2 + p8*home2 + p9*top2

    shb_finger1_col_pen = p10*is_pinky1 + p11*is_ring1 + p12*is_middle1 + p13*is_index1
    shb_finger2_col_pen = p14*is_pinky2 + p15*is_ring2 + p16*is_middle2 + p17*is_index2
    shb_row_pen = (shb_finger1_row_pen + shb_finger2_row_pen)
    shb_col_pen = (shb_finger1_col_pen + shb_finger2_col_pen)

    shb_finger_pen = shb_row_pen * shb_col_pen

    # alt weighting
    alt_finger1_row_pen = p18*bottom1 + p19*home1 + p20*top1
    alt_finger2_row_pen = p21*bottom2 + p22*home2 + p23*top2

    alt_finger1_col_pen = p24*is_pinky1 + p25*is_ring1 + p26*is_middle1 + p27*is_index1
    alt_finger2_col_pen = p28*is_pinky2 + p29*is_ring2 + p30*is_middle2 + p31*is_index2
    alt_finger1_row_pen = (alt_finger1_row_pen + alt_finger2_row_pen)
    alt_finger1_col_pen = (alt_finger1_col_pen + alt_finger2_col_pen)

    alt_finger_pen = alt_finger1_row_pen * alt_finger1_col_pen

    # Finger weighting
    sfb_finger1_row_pen = p32*bottom1 + p33*home1 + p34*top1
    sfb_finger2_row_pen = p35*bottom2 + p36*home2 + p37*top2

    sfb_finger1_col_pen = p38*is_pinky1 + p39*is_ring1 + p40*is_middle1 + p41*is_index1
    sfb_finger2_col_pen = p42*is_pinky2 + p43*is_ring2 + p44*is_middle2 + p45*is_index2
    sfb_finger1_row_pen = (sfb_finger1_row_pen + sfb_finger2_row_pen)
    sfb_finger1_col_pen = (sfb_finger1_col_pen + sfb_finger2_col_pen)

    sfb_finger_pen = sfb_finger1_row_pen * sfb_finger1_col_pen
    
    # Base finger pen
    base_pen = (p46*bottom2 + p47*home2 + p48*top2) * (p49*is_pinky2 + p50*is_ring2 + p51*is_middle2 + p52*is_index2)

    # BG Type Classification
    same_hand_weight = same_hand*(1-is_sfb)*shb_finger_pen
    sfb_weight = is_sfb*sfb_finger_pen
    alt_weight = (1-same_hand)*alt_finger_pen

    return  (same_hand_weight + sfb_weight + alt_weight + base_pen) * freq_pen


"""def model_function(features, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, 
                   p21, p22, p23, p24, p25, p26, p27, p28, p29, p30, p31, p32, p33, p34, p35, p36, p37, p38, p39, p40, 
                   p41, p42, p43, p44, p45, p46, p47, p48, p49, p50, p51, p52, p53, p54, p55, p56, p57, p58, p59, p60, 
                   p61, p62, p63, p64, p65, p66, p69, p70, p71, p73, p74, p75, p76, p77, p78, p79): #  p67, p68,p70,  p71, p72,
    freq, is_sfb, dx, dy, same_hand = features[0], *features[9:13]
    bottom1, home1, top1, bottom2, home2, top2 = features[13:19]
    freq_pen = (p1 * np.log(freq + p2) + p3)
    
    row_pen1 = (p4 * bottom2 + p5 * home2 + p6 * top2 + p7)
    row_pen2 = (p8 * bottom2 + p9 * home2 + p10 * top2 + p11)
    row_pen2b = (p12 * bottom1 + p13 * home1 + p14 * top1 + p15)
    row_pen3 = (p16 * bottom2 + p17 * home2 + p18 * top2 + p19)
    row_pen3b = (p20 * bottom1 + p21 * home1 + p22 * top1 + p23)
    row_pen4 = (p24 * bottom2 + p25 * home2 + p26 * top2 + p27)
    row_pen4b = (p28 * bottom1 + p29 * home1 + p30 * top1 + p31)

    col_pen1 = (p32 * features[5] + p33 * features[6] + p34 * features[7] + p35 * features[8] + p36)
    col_pen2 = (p37 * features[5] + p38 * features[6] + p39 * features[7] + p40 * features[8] + p41)
    col_pen2b = (p42 * features[1] + p43 * features[2] + p44 * features[3] + p45 * features[4] + p46)
    col_pen3 = (p47 * features[5] + p48 * features[6] + p49 * features[7] + p50 * features[8] + p51)
    col_pen3b = (p52 * features[1] + p53 * features[2] + p54 * features[3] + p55 * features[4] + p56)
    col_pen4 = (p57 * features[5] + p58 * features[6] + p59 * features[7] + p60 * features[8] + p61)
    col_pen4b = (p62 * features[1] + p63 * features[2] + p64 * features[3] + p65 * features[4] + p66)

    # finger_w1 =  + p67
    dist = (dx ** 2 + dy ** 2) ** 0.5
    finger_w2 = col_pen2 * row_pen2 * row_pen2b * col_pen2b
    finger_w3 = col_pen3 * row_pen3 * row_pen3b * col_pen3b + p69
    
    sfb_pen = (p70 * is_sfb * dist + p71)
    scissor_pen = (p73 * (1 - is_sfb) * same_hand * (p74 + dy * p75) * finger_w2 + p76) # 
    sfb_boost = (p77 * (1 - same_hand) * finger_w3 + p78)
    finger_pen = col_pen1 * row_pen1 + col_pen4 * col_pen4b * row_pen4 * row_pen4b

    return freq_pen * sfb_boost * scissor_pen * sfb_pen * finger_pen

sfb mode
def model_function(features, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, t, u, v, w, x, y, z, aa, ab, ac, ad, ae, af, ag, ah, aj, ai, ak, al, am, an): # 
    freq, is_sfb, dx, dy, same_hand = features[0], features[9], features[10], features[11], features[12]
    bottom, home, top = features[13:16]
    freq_pen = (a * np.log(freq + b) + c)
    
    # sfb_finger_pen = (d*features[5]+e*features[6]+f*features[7]+g*features[8] + h)
    row_pen1 = (d*bottom+e*home+f*top+g)
    row_pen2 = (h*bottom+aa*home+ab*top+ac)
    row_pen3 = (ai*bottom+aj*home+ak*top+al)

    finger_pen1 = (t*features[5]+u*features[6]+v*features[7]+w*features[8] + x)
    finger_pen2 = (k*features[5]+l*features[6]+m*features[7]+n*features[8] + o)
    finger_pen3 = (ad*features[5]+ae*features[6]+af*features[7]+ag*features[8] + ah)
    #finger_pen = finger_pen1+finger_pen2

    dist = (dx**2+dy**2)**0.5
    sfb_pen = (i*is_sfb*dist*(finger_pen1+row_pen1)+j)
    scissor_pen = ((1-is_sfb)*p*(1/(dx+r))*(finger_pen2+row_pen2)*same_hand*(al+dy*am)+q)
    sfb_boost = y*(1-same_hand)*(finger_pen3+row_pen3)+z

    return freq_pen+sfb_boost+sfb_pen+scissor_pen
"""

r2 = 0

while r2 < 0.44:
    initial_guess = (np.random.random(52)*2-1)
    popt, pcov = curve_fit(model_function, input_data, times, method="trf", maxfev=750000) # "trf" p0=initial_guess

    sum_of_squares = np.sum((times - np.mean(times))**2)

    new_y = model_function(input_data, *popt)
    residuals = times-new_y
    r2 = 1 - np.sum((residuals)**2)/sum_of_squares

    print("R^2:", r2)
    print("MAE:", np.mean(np.abs(residuals)))

# What do these errors mean?
# print("Param Errors:", np.sqrt(np.diag(pcov)))

print(list(popt))
for i, v in enumerate(popt):
    print(f"p{i}:", v)

/tmp/ipykernel_8200/2176078543.py:24: RuntimeWarning: invalid value encountered in log
  freq_pen = (p1 * np.log(freq + p2) + p3)


R^2: 0.4400781426958873
MAE: 40.48073291166524
[-0.25270784040533245, -14.974348452997553, 21.545808832550467, 4.241640945972414, 4.061331723603196, 4.124519503249249, 5.467495178099448, 3.3162393938359815, 3.646854202077939, 1.6361356739091275, 1.4402901576946792, 1.3765412174551757, 1.3118479760187225, -5.372970594701722, 0.47424528856822784, 2.65189161754224, 8.04193414013673, 4.214536700108369, 4.0028404142901906, 4.144119570608414, 5.404146964731018, 3.383041258981667, 3.5777187643154105, 1.5318604690011148, 1.3552466348871341, 1.4246559378312806, 1.3278468511926815, -5.325650583820584, 0.3402077404098995, 2.627847510541285, 7.994579742838234, 3.581769538536918, 3.397505771688884, 3.5992554332408977, 4.702062892697479, 2.8018720828080785, 3.0750571454237705, -2.1113602877629822, 1.0166912350351927, 2.341002252075287, 5.398882264348432, -2.1113602877273774, 1.0166912348030903, 2.3410022521842735, 5.398882264367258, 8.280727640668418, 5.828866248174598, 6.376179101626531, 8.28890466

In [ ]:
def model_function(features, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, p24, p25, p26, p27, p28, p29, p30,
                   p31, p32, p33, p34, p35, p36, p37, p38, p39, p40, p41, p42, p43, p44, p45, p46, p47, p48, p49, p50, p51, p52):
    freq, is_sfb, is_lateral, dx, dy, same_hand = features[0], *features[9:14]
    
    #input_data = [freqs, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, is_sfb, is_lateral, delta_x, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2]

    bottom1, home1, top1, bottom2, home2, top2 = features[14:20]
    is_pinky1, is_ring1, is_middle1, is_index1 = features[1:5]
    is_pinky2, is_ring2, is_middle2, is_index2 = features[5:9]

    freq_pen = (p1 * np.log(freq + p2) + p3)
    
    # SHB weighting
    shb_finger1_row_pen = p4*bottom1 + p5*home1 + p6*top1
    shb_finger2_row_pen = p7*bottom2 + p8*home2 + p9*top2

    shb_finger1_col_pen = p10*is_pinky1 + p11*is_ring1 + p12*is_middle1 + p13*is_index1
    shb_finger2_col_pen = p14*is_pinky2 + p15*is_ring2 + p16*is_middle2 + p17*is_index2
    shb_row_pen = (shb_finger1_row_pen + shb_finger2_row_pen)
    shb_col_pen = (shb_finger1_col_pen + shb_finger2_col_pen)

    shb_finger_pen = shb_row_pen * shb_col_pen

    # alt weighting
    alt_finger1_row_pen = p18*bottom1 + p19*home1 + p20*top1
    alt_finger2_row_pen = p21*bottom2 + p22*home2 + p23*top2

    alt_finger1_col_pen = p24*is_pinky1 + p25*is_ring1 + p26*is_middle1 + p27*is_index1
    alt_finger2_col_pen = p28*is_pinky2 + p29*is_ring2 + p30*is_middle2 + p31*is_index2
    alt_finger1_row_pen = (alt_finger1_row_pen + alt_finger2_row_pen)
    alt_finger1_col_pen = (alt_finger1_col_pen + alt_finger2_col_pen)

    alt_finger_pen = alt_finger1_row_pen * alt_finger1_col_pen

    # Finger weighting
    sfb_finger1_row_pen = p32*bottom1 + p33*home1 + p34*top1
    sfb_finger2_row_pen = p35*bottom2 + p36*home2 + p37*top2

    sfb_finger1_col_pen = p38*is_pinky1 + p39*is_ring1 + p40*is_middle1 + p41*is_index1
    sfb_finger2_col_pen = p42*is_pinky2 + p43*is_ring2 + p44*is_middle2 + p45*is_index2
    sfb_finger1_row_pen = (sfb_finger1_row_pen + sfb_finger2_row_pen)
    sfb_finger1_col_pen = (sfb_finger1_col_pen + sfb_finger2_col_pen)

    sfb_finger_pen = sfb_finger1_row_pen * sfb_finger1_col_pen
    
    # Base finger pen
    base_pen = (p46*bottom2 + p47*home2 + p48*top2) * (p49*is_pinky2 + p50*is_ring2 + p51*is_middle2 + p52*is_index2)

    # BG Type Classification
    same_hand_weight = same_hand*(1-is_sfb)*shb_finger_pen
    sfb_weight = is_sfb*sfb_finger_pen
    alt_weight = (1-same_hand)*alt_finger_pen

    return  (same_hand_weight + sfb_weight + alt_weight + base_pen) * freq_pen

In [6]:
# Validate
row_names = "freqs, is_pinky1, is_ring1, is_middle1, is_index1, is_pinky2, is_ring2, is_middle2, is_index2, is_sfb, is_lateral, delta_x, y_dist, same_hand, bottom1, home1, top1, bottom2, home2, top2".split(", ")
k = 5
n = input_data.shape[1]

# x-axis == actual
# y-axis == predicted
# mean absolute percent error
# predicted v.s. actual plot 
for _ in range(5): # random restarts
    order = 1+np.random.permutation(n-1)
    shuffled_input = input_data[:, order]
    shuffled_times = times[order]

    folds = np.array_split(shuffled_input,k,axis=1)
    time_folds = np.array_split(shuffled_times,k)

    maes = []
    r2s = []

    print(times[0])

    for i in range(k):
        training_set = np.concatenate([input_data[:, 0:1], np.concatenate(folds[:i]+folds[i+1:],axis=1)], axis=1)
        training_times = np.concatenate([[times[0]]]+time_folds[:i]+time_folds[i+1:])
        
        validation_set = np.concatenate([input_data[:, 0:1], folds[i]], axis=1)
        validation_times = np.concatenate([[times[0]],time_folds[i]])

        popt, pcov = curve_fit(model_function, training_set, training_times, method="trf", maxfev=750000) # "trf" p0=initial_guess
        sum_of_squares = np.sum((training_times - np.mean(training_times))**2)
        # sum_of_squares = np.sum((times - np.mean(times))**2)

        predicted_times = model_function(validation_set, *popt)
        residuals = validation_times-predicted_times
        r2 = 1 - np.sum((residuals)**2)/sum_of_squares
        mae = np.mean(np.abs(residuals))
        r2s.append(r2)
        maes.append(mae)

        if np.isnan(r2):
            p1, p2, p3 = popt[:3]
            print(np.min(validation_set[0,:]))
            #print(training_set[0])
            trouble_col = validation_set[:, np.isnan(predicted_times)]
            #print(trouble_col)
            print("output:", model_function(trouble_col, *popt))
            print("freq", p1 * np.log(trouble_col[0] + p2) + p3, "cuz", popt[:3])
            #for v in training_set[0]:
            #    p1, p2, p3 = popt[:3]
            #    res = (p1 * np.log(v + p2) + p3)
            break

        print(f"Fold {i+1}")
        print("R^2:", r2)
        print("MAE:", mae)
        print()
        # print("MSE:", np.mean(residuals**2))

print("Average R^2", np.mean(r2s))
print("Average MAE", np.mean(maes))
    

478.7230483271375


/tmp/ipykernel_8200/2176078543.py:24: RuntimeWarning: invalid value encountered in log
  freq_pen = (p1 * np.log(freq + p2) + p3)


Fold 1
R^2: 0.8708671501373083
MAE: 40.33080004648577



KeyboardInterrupt: 

In [7]:
%matplotlib qt

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

diverging = ['#ff0055', '#ff1959', '#ff265c', '#ff305f', '#ff3963', '#ff4066', '#ff4769', '#ff4d6d', '#ff5370', '#ff5973', '#ff5e77', '#ff647a', '#ff697e', '#ff6e81', '#ff7284', '#ff7788', '#ff7c8b', '#ff808e', '#ff8592', '#ff8995', '#ff8d99', '#ff929c', '#ff96a0', '#ff9aa3', '#ff9ea6', '#ffa2aa', '#ffa6ad', '#ffaab1', '#ffaeb4', '#ffb2b8', '#ffb6bb', '#ffbabf', '#ffbec2', '#ffc2c6', '#ffc6c9', '#ffcacd', '#ffced0', '#ffd1d4', '#ffd5d7', '#ffd9db', '#ffdddf', '#ffe1e2', '#ffe5e6', '#ffe8e9', '#ffeced', '#fff0f1', '#fff4f4', '#fff7f8', '#fffbfb', '#ffffff', '#fbfdfe', '#f7fbfc', '#f3f9fb', '#eff8f9', '#ebf6f8', '#e7f4f7', '#e3f2f5', '#dff0f4', '#dbeef2', '#d7ecf1', '#d3ebf0', '#cfe9ee', '#cbe7ed', '#c7e5eb', '#c3e3ea', '#bfe1e9', '#bbdfe7', '#b7dee6', '#b3dce4', '#afdae3', '#abd8e2', '#a6d6e0', '#a2d4df', '#9ed2dd', '#9ad1dc', '#96cfda', '#91cdd9', '#8dcbd8', '#89c9d6', '#84c7d5', '#80c5d3', '#7bc3d2', '#77c2d1', '#72c0cf', '#6ebece', '#69bccc', '#64bacb', '#5fb8c9', '#5ab6c8', '#54b4c7', '#4fb2c5', '#49b1c4', '#43afc2', '#3cadc1', '#35abc0', '#2da9be', '#23a7bd', '#17a5bb', '#00a3ba']
rainbow = ['#800000', '#81000e', '#820119', '#830222', '#84042a', '#850631', '#850938', '#850c3f', '#851046', '#85134c', '#851653', '#841959', '#831c5f', '#821f65', '#81226c', '#802571', '#7f2877', '#7d2a7d', '#7b2d82', '#793088', '#77338d', '#753692', '#723997', '#6f3c9b', '#6c3fa0', '#6942a4', '#6545a7', '#6248ab', '#5e4bae', '#594eb1', '#5451b4', '#4f54b6', '#4a57b8', '#445bba', '#3d5ebb', '#3561bc', '#2b64bd', '#1f67bd', '#0b6bbd', '#006ebd', '#0071bc', '#0074bb', '#0077ba', '#007ab8', '#007db6', '#0080b4', '#0083b2', '#0085af', '#0088ac', '#008aa9', '#008da5', '#008fa2', '#00919e', '#00939a', '#009595', '#009791', '#00998c', '#009a88', '#009b83', '#009d7e', '#009e79', '#009f73', '#00a06e', '#00a068', '#00a163', '#1da15d', '#31a257', '#40a251', '#4ca24b', '#57a245', '#61a23f', '#6ba138', '#74a131', '#7ca02a', '#84a022', '#8c9f19', '#949e0d', '#9b9d00', '#a29c00', '#a99b00', '#af9a00', '#b69900', '#bc9800', '#c29700', '#c79600', '#cd9400', '#d29300', '#d79210', '#db911c', '#e09026', '#e48f2e', '#e88e37', '#ec8d3f', '#ef8c47', '#f38b4f', '#f68b56', '#f88a5e', '#fb8a66', '#fd8a6d']
magma = ['#000a1e', '#000c22', '#000d27', '#000f2b', '#001030', '#001134', '#001338', '#01143d', '#021541', '#031646', '#05174a', '#08194f', '#0b1a53', '#0e1b57', '#121c5c', '#151d60', '#191e64', '#1c1f68', '#20206d', '#242171', '#272274', '#2b2278', '#2f237c', '#332480', '#372583', '#3c2687', '#40278a', '#44288d', '#492890', '#4d2993', '#522a95', '#572b98', '#5b2c9a', '#602d9c', '#652e9e', '#6a2ea0', '#6f2fa2', '#7430a3', '#7931a4', '#7e32a5', '#8433a6', '#8934a7', '#8e35a7', '#9336a7', '#9937a7', '#9e38a7', '#a339a7', '#a83ba6', '#ae3ca6', '#b33da5', '#b83fa3', '#bd40a2', '#c242a0', '#c7439f', '#cc459d', '#d1479b', '#d64999', '#da4b96', '#df4e94', '#e35091', '#e8538e', '#ec568c', '#f05889', '#f45c85', '#f75f82', '#fb627f', '#fe667c', '#ff6a79', '#ff6e75', '#ff7272', '#ff766f', '#ff7b6b', '#ff7f68', '#ff8465', '#ff8962', '#ff8d60', '#ff925e', '#ff975c', '#ff9d5a', '#ffa259', '#ffa759', '#ffac59', '#ffb25a', '#ffb75b', '#ffbc5d', '#ffc160', '#ffc764', '#ffcc68', '#ffd16d', '#ffd672', '#ffdb78', '#ffe07f', '#ffe586', '#ffea8d', '#ffee94', '#fef39c', '#fcf7a4', '#fbfbac', '#f9ffb4']
viv = list(reversed(['#fff829', '#f4f730', '#e8f737', '#ddf63f', '#d1f546', '#c5f34d', '#b9f254', '#acf15b', '#a0ef62', '#92ed68', '#85ec6e', '#76ea74', '#66e77a', '#55e580', '#41e385', '#24e08b', '#00dd90', '#00da95', '#00d799', '#00d49e', '#00d1a2', '#00cda6', '#00caaa', '#00c6ae', '#00c2b1', '#00beb4', '#00bab7', '#00b6ba', '#00b2bc', '#00adbf', '#00a9c0', '#00a4c2', '#00a0c3', '#009bc4', '#0096c5', '#0091c6', '#008dc6', '#0088c6', '#0083c5', '#007ec4', '#0079c3', '#0074c2', '#006fc0', '#0069be', '#0064bc', '#005fb9', '#145ab6', '#2155b3', '#2950b0', '#304bac', '#3547a8', '#3942a4', '#3d3d9f', '#40389b', '#423396', '#442f91', '#462a8b', '#472586', '#482080', '#491c7a', '#491775', '#49116f', '#490c68', '#490562']))
mycmap = LinearSegmentedColormap.from_list('custom_colormap', magma)

plt.figure()

display = np.arange(len(freqs)) # freqs
scatter = plt.scatter(display, times, c=c, s=50)

for x, y, label in zip(display, times, labels):
    plt.annotate(label, (x,y))

plt.plot(display, new_y, c="black")
#plt.axhline(0, color='black', linewidth=0.5)
#plt.scatter(freqs, times-new_y, c="red")
plt.xlabel("Frequency Index")
plt.ylabel("Average Typing Time (Milliseconds)")
#plt.xscale("log")

plt.show()


In [8]:
plt.figure()

scatter = plt.scatter(new_y, times, c=c, s=(freqs/np.max(freqs))*1000)
plt.plot()

x_line = np.linspace(min(min(new_y),min(times)), max(max(new_y),max(times)), 2)

#for x, y, label in zip(display, times, labels):
#    plt.annotate(label, (x,y))

#plt.axhline(0, color='black', linewidth=0.5)
#plt.scatter(freqs, times-new_y, c="red")
plt.plot(x_line, x_line, c="black")
plt.xlabel("Predicted Typing Time (Milliseconds)")
plt.ylabel("Actual Typing Time (Milliseconds)")
#plt.xscale("log")

plt.show()


In [9]:
plt.figure()

# Add text annotations
for i, pos in enumerate(display):
    plt.text(pos, np.median(actual_times[i]), labels[i], ha='center', va='bottom', color='black', fontsize=11)

# Create a box plot
boxplot = plt.boxplot(actual_times, positions=display, patch_artist=True, showfliers=False)

for i, box in enumerate(boxplot['boxes']):
    box.set(facecolor=c[i])  # Set the desired color here

# Set labels
plt.xlabel("Frequency Index")
plt.ylabel("Average Typing Time (Milliseconds)")

plt.show()

In [10]:
import numpy as np
import matplotlib.pyplot as plt

# Plotting the covariance matrix as a heatmap
print(pcov)

# sorted_pcov = pcov[np.argsort(np.sum(np.clip(pcov, 0, None), axis=0))]

def symmetric_log_transform(matrix):
    sign_matrix = np.sign(matrix)
    log_matrix = np.log(np.abs(matrix)*0.0000000000001 + 1)
    return sign_matrix * log_matrix

plt.figure(figsize=(6, 6))
plt.imshow((pcov), cmap=mycmap, interpolation='nearest')
plt.colorbar(label='Covariance')
plt.title('Normalized Covariance Matrix')

# Set tick positions and labels
tick_positions = np.arange(pcov.shape[0])
tick_labels = np.arange(1, pcov.shape[0] + 1)

plt.xticks(tick_positions, tick_labels)
plt.yticks(tick_positions, tick_labels)

# Adjust the ticks and gridlines to be at the center of the cells
plt.gca().set_xticks(np.arange(-0.5, pcov.shape[0], 1), minor=True)
plt.gca().set_yticks(np.arange(-0.5, pcov.shape[1], 1), minor=True)
plt.grid(which='minor', color='k', linestyle='-', linewidth=1)

plt.xlabel('Parameters')
plt.ylabel('Parameters')
plt.show()

[[ 2.81042609e+11 -4.21161685e+06 -5.66390021e+12 ... -1.65064666e+12
  -1.72244913e+12 -2.53153665e+12]
 [-4.21161685e+06  3.93855938e+04  8.48774284e+07 ...  2.74323648e+08
   2.86256600e+08  4.20720154e+08]
 [-5.66390021e+12  8.48774284e+07  1.14145559e+14 ...  3.32657673e+13
   3.47128148e+13  5.10184954e+13]
 ...
 [-1.65064666e+12  2.74323648e+08  3.32657673e+13 ...  1.03950879e+14
   1.08472700e+14  1.59425676e+14]
 [-1.72244913e+12  2.86256600e+08  3.47128148e+13 ...  1.08472700e+14
   1.13191219e+14  1.66360629e+14]
 [-2.53153665e+12  4.20720154e+08  5.10184954e+13 ...  1.59425676e+14
   1.66360629e+14  2.44505350e+14]]
